In [1]:
%defaultDatasource jdbc:h2:mem:db

In [62]:
DROP TABLE IF EXISTS proteinAtlas;
DROP TABLE IF EXISTS proteinAtlas_entry_pathologyExpression_data;
DROP TABLE IF EXISTS rnaExpression;
DROP TABLE IF EXISTS proteinAtlas_entry_rnaExpression_data;
DROP TABLE IF EXISTS RNASample;

create table proteinAtlas (
    PK_proteinAtlas INTEGER,
    FILENAME VARCHAR,
    FILEPATH VARCHAR,
    copyright VARCHAR,
    entry_cellExpression_data_location VARCHAR,
    entry_cellExpression_data_location_GOId VARCHAR,
    entry_cellExpression_data_location_status VARCHAR,
    entry_cellExpression_image_imageType VARCHAR,
    entry_cellExpression_image_imageUrl VARCHAR,
    entry_cellExpression_source VARCHAR,
    entry_cellExpression_summary VARCHAR,
    entry_cellExpression_technology VARCHAR,
    entry_cellExpression_verification VARCHAR,
    entry_cellExpression_verification_type VARCHAR,
    entry_identifier_db VARCHAR,
    entry_identifier_id VARCHAR,
    entry_identifier_version DOUBLE,
    entry_identifier_xref_db VARCHAR,
    entry_identifier_xref_id VARCHAR,
    entry_name VARCHAR,
    entry_pathologyExpression_assayType VARCHAR,
    entry_pathologyExpression_rnaCancerDistribution VARCHAR,
    entry_pathologyExpression_rnaCancerSpecificity VARCHAR,
    entry_pathologyExpression_source VARCHAR,
    entry_pathologyExpression_technology VARCHAR,
    entry_proteinEvidence_evidence VARCHAR,
    entry_synonym VARCHAR,
    entry_tissueExpression_assayType VARCHAR,
    entry_tissueExpression_source VARCHAR,
    entry_tissueExpression_summary VARCHAR,
    entry_tissueExpression_summary_type VARCHAR,
    entry_tissueExpression_technology VARCHAR,
    entry_tissueExpression_verification VARCHAR,
    entry_tissueExpression_verification_description VARCHAR,
    entry_tissueExpression_verification_type VARCHAR,
    entry_url VARCHAR,
    entry_version INTEGER,
    schemaVersion DOUBLE,
    
    PRIMARY KEY(PK_proteinAtlas)    
) AS SELECT
    PK_proteinAtlas,
    FILENAME,
    FILEPATH,
    copyright,
    entry_cellExpression_data_location,
    entry_cellExpression_data_location_GOId,
    entry_cellExpression_data_location_status,
    entry_cellExpression_image_imageType,
    entry_cellExpression_image_imageUrl,
    entry_cellExpression_source,
    entry_cellExpression_summary,
    entry_cellExpression_technology,
    entry_cellExpression_verification,
    entry_cellExpression_verification_type,
    entry_identifier_db,
    entry_identifier_id,
    entry_identifier_version,
    entry_identifier_xref_db,
    entry_identifier_xref_id,
    entry_name,
    entry_pathologyExpression_assayType,
    entry_pathologyExpression_rnaCancerDistribution,
    entry_pathologyExpression_rnaCancerSpecificity,
    entry_pathologyExpression_source,
    entry_pathologyExpression_technology,
    entry_proteinEvidence_evidence,
    entry_synonym,
    entry_tissueExpression_assayType,
    entry_tissueExpression_source,
    entry_tissueExpression_summary,
    entry_tissueExpression_summary_type,
    entry_tissueExpression_technology,
    entry_tissueExpression_verification,
    entry_tissueExpression_verification_description,
    entry_tissueExpression_verification_type,
    entry_url,
    entry_version,
    schemaVersion
FROM CSVREAD('./proteinAtlas.csv');

create table proteinAtlas_entry_pathologyExpression_data (
    FK_proteinAtlas INTEGER,
    survivalAnalysis_dataSource VARCHAR,
    survivalAnalysis_isPrognostic BOOLEAN,
    survivalAnalysis_prognosticType VARCHAR,
    survivalAnalysis_pValue DOUBLE,
    survivalAnalysis_source VARCHAR,
    tissue VARCHAR,
    tissue_organ VARCHAR,
    
    FOREIGN KEY(FK_proteinAtlas) REFERENCES proteinAtlas(PK_proteinAtlas)
) AS SELECT
    FK_proteinAtlas,
    survivalAnalysis_dataSource,
    survivalAnalysis_isPrognostic,
    survivalAnalysis_prognosticType,
    survivalAnalysis_pValue,
    survivalAnalysis_source,
    tissue,
    tissue_organ
FROM CSVREAD('./proteinAtlas_entry_pathologyExpression_data.csv');

create table rnaExpression (
    PK_rnaExpression INTEGER,
    FK_proteinAtlas INTEGER,
    assayType VARCHAR,
    rnaDistribution VARCHAR,
    rnaDistribution_description VARCHAR,
    rnaSpecificity_description VARCHAR,
    rnaSpecificity_specificity VARCHAR,
    rnaSpecificity_tissue VARCHAR,
    rnaSpecificity_tissue_ontologyTerms VARCHAR,
    rnaSpecificity_tissue_organ VARCHAR,
    source VARCHAR,
    technology VARCHAR,
    
    PRIMARY KEY(PK_rnaExpression),
    FOREIGN KEY(FK_proteinAtlas) REFERENCES proteinAtlas(PK_proteinAtlas)
) AS SELECT
    PK_rnaExpression,
    FK_proteinAtlas,
    assayType,
    rnaDistribution,
    rnaDistribution_description,
    rnaSpecificity_description,
    rnaSpecificity_specificity,
    rnaSpecificity_tissue,
    rnaSpecificity_tissue_ontologyTerms,
    rnaSpecificity_tissue_organ,
    source,
    technology
FROM CSVREAD('./rnaExpression.csv');

create table proteinAtlas_entry_rnaExpression_data (
    PK_proteinAtlas_entry_rnaExpression_data INTEGER,
    FK_rnaExpression INTEGER,
    bloodCell VARCHAR,
    bloodCell_lineage VARCHAR,
    cellLine VARCHAR,
    cellLine_cellosaurusID VARCHAR,
    cellLine_organ VARCHAR,
    tissue VARCHAR,
    tissue_ontologyTerms VARCHAR,
    tissue_organ VARCHAR,
    tissue_region VARCHAR,
    
    PRIMARY KEY(PK_proteinAtlas_entry_rnaExpression_data),
    FOREIGN KEY(FK_rnaExpression) REFERENCES rnaExpression(PK_rnaExpression)
) AS SELECT
    PK_proteinAtlas_entry_rnaExpression_data,
    FK_rnaExpression,
    bloodCell,
    bloodCell_lineage,
    cellLine,
    cellLine_cellosaurusID,
    cellLine_organ,
    tissue,
    tissue_ontologyTerms,
    tissue_organ,
    tissue_region
FROM CSVREAD('./proteinAtlas_entry_rnaExpression_data.csv');

create table RNASample (
    FK_data INTEGER,
    age INTEGER,
    expRNA DOUBLE,
    sampleId INTEGER,
    sex VARCHAR,
    unitRNA VARCHAR,
    
    PRIMARY KEY(sampleId),
    FOREIGN KEY(FK_data) REFERENCES proteinAtlas_entry_rnaExpression_data(PK_proteinAtlas_entry_rnaExpression_data)
) AS SELECT
    FK_data,
    age,
    expRNA,
    sampleId,
    sex,
    unitRNA
FROM CSVREAD('./RNASample.csv');

## Dadas as patologias, quais destas possuem amostras de RNA dadas por pessoas com mais de 60 anos?

In [63]:


select  --RNASample.sampleId,
        --RNASample.age,
        --RNASample.sex,
        distinct
            pathology.tissue
        
from RNASample RNASample
JOIN proteinAtlas_entry_rnaExpression_data rnaExpressionData ON RNASample.FK_DATA = rnaExpressionData.PK_proteinAtlas_entry_rnaExpression_data
JOIN rnaExpression rnaExpression ON rnaExpression.PK_rnaExpression = rnaExpressionData.FK_rnaExpression
JOIN proteinAtlas pa ON pa.PK_proteinAtlas = rnaExpression.FK_proteinAtlas
JOIN proteinAtlas_entry_pathologyExpression_data pathology ON pathology.tissue_organ = rnaExpressionData.tissue_organ

group by RNASample.age, pathology.tissue
having RNASample.age > 80
;

In [43]:
select * from proteinAtlas_entry_pathologyExpression_data;